**This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm.**

*Step 1:* Training the model with set of emails labelled as either from Spam or Not Spam. There are 702 emails equally divided into spam and non spam category. 

*Step 2:*  Test the model on 260 emails. We shall ask model to predict the category of this emails and compare the accuracy with correct classification that we already know

Assumptions: independence in features 

In [54]:
#Step 1 Import packages 
# -*- coding: utf-8 -*-

import os # Miscellaneous operating system interfaces, provides a portable way of using operating system dependent functionality
import numpy as np # Numerical Python, to work with large, multi-dimensional arrays and matrices,high-level mathematical functions to operate on arrays
from collections import Counter #dict subclass for counting hashable objects
from sklearn.naive_bayes import GaussianNB #implements the Gaussian Naive Bayes algorithm for classification
from sklearn.metrics import accuracy_score #multilabel classification, computes subset accuracy


In [55]:
# simple way to share files is to mount your Google Drive
from google.colab import drive 
drive.mount('/content/drive',force_remount=True)

# allow access to drive and enter authentication code 

Mounted at /content/drive


This function builds a Dictionary of most common 3000 words from all the email content. 
First it adds all words and symbols in the dictionary. 
Then it removes all non-alpha-numeric characters and any single character alpha-numeric characters. 
After this is complete it shrinks the Dictionary by keeping only most common 3000 words in the dictionary. It returns the Dictionary.

In [56]:
#Function that reads the email files from a folder and constructs a dictionary for all words

def make_dict(root_dir):  # defining the function, with parameter ""
  each_word = [] # creating empty list 

  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] # creating a list 'emails', 
  # os.path.join() method returns a string which represents the concatenated path components
  # os.listdir() method retrieves a list of all the files in ''
  # for loop through all emails in list_of_emails (change line above)
  for mail in emails:  
    with open(mail) as m:  #'with' simplifies management file streams, no need to call file.close(), save as ''
      for line in m: # loop through each item in email 
        words = line.split() # returns a list of all the words in the string
        each_word += words # creating list with all words (addition)
  dictionary = Counter(each_word) # creates dictionary with key + value (keeps track of how many times equivalent values are added)
  list_to_remove = list(dictionary) # returns a list from dictionary 

  for item in list_to_remove:  # iterating over the list 
    if item.isalpha() == False: # deleting if not alphabet letter (a-z), isaplha()- boolean True/False 
      del dictionary[item]
    elif len(item) == 1: # deleting if only 1 character (lenght of word = 1)
      del dictionary[item]
  dictionary = dictionary.most_common(3000) # returns list,sorted based on the count of the elements, limit 3000 and saving back in dictionary
  return dictionary

This function extracts feature columns and populates their values (Feature Matrix of 3000 comumns and rows equal to the number of email files). 
The function also analyzes the File Names of each email file and decides if it's a Spam or not based on the naming convention. 
Based on this the function also creates the Labelled Data Column. This function is used to extract the training dataset as well as the testing dataset and returns the Feature Dataset and the Label column.


In [57]:


# extract features function that removes less common words from every document 

def extr_features(mail_dir): # defining fucntion with parameter ''
  all_files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)] # create list with joined path components 

  feat_matrix = np.zeros((len(all_files),3000)) # Return a new array (Matrix) of given shape(len of files-rows) and columns 3000 (and optional type), filled with zeros
  train_labels = np.zeros(len(all_files)) # zeros() function --> matrix with 0 with give shape (len of files)
  count = 1; # start of counter set to 1
  docID = 0; #docId set to 0 
  for fil in all_files: # iterate over each file 
    with open(fil) as fi: # save as fi 
      for i, line in enumerate(fi): # enumerate to get counter in loop
        if i ==2: 
          words = line.split() # split if 2 
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                feat_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/') # use split method at each character backslash ('/') character 
      lastToken = filepathTokens[len(filepathTokens)-1]

      if lastToken.startswith("spmsg"):  #  we know that spmsga162.txt files are spam 
                                         #
        train_labels[docID] = 1; # label as 1 (spam mail), dummy variable?!
        count = count + 1 # increaese by 1 for each loop 
      docID = docID + 1 
  return feat_matrix, train_labels


The section is the main Program that calls the above two functions and gets executed first. 
First it "trains" the model using model.fit function and Training Dataset. 
After that it scores the Test Data set by running the Trained Model with the Test Data set. 
At the end it prints the model performance in terms of accuracy score.


In [58]:
TRAIN_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/train-mails'
TEST_DIR = '/content/drive/My Drive/MSBA_Colab_2020/ML_Algorithms/CA02/Data/test-mails'

In [60]:

dictionary = make_dict(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
feat_matrix, labels = extr_features(TRAIN_DIR)
test_feat_matrix, test_labels = extr_features(TEST_DIR)

model = GaussianNB()

print ("Training Model using Gaussian Naibe Bayes algorithm .....")
# train model
model.fit(feat_matrix, labels)
print ("Training completed")
print ("testing trained model to predict Test Data labels")
#predict/test
predicted_labels = model.predict(test_feat_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")
print (accuracy_score(test_labels, predicted_labels))
# percentage of correct predictions 

"""======================= END OF PROGRAM ========================="""

FileNotFoundError: ignored